In [ ]:
#Importando Bibliotecas necessarias
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2
import time



### WebScraping coin market cap

In [ ]:
# Configuração do Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://coinmarketcap.com/")

# Aguarda a página carregar
driver.implicitly_wait(10)

# Extrai o conteúdo da página
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# Extrai os dados da tabela
table = soup.find('table')

data = []
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if cols:
        data.append(cols)

# Filtrando linhas para que seja usado o número correto de colunas (10 neste caso)
filtered_data = [row for row in data if len(row) == 11]

# Criar o DataFrame com as colunas corretas
df = pd.DataFrame(filtered_data, columns=[
    '', 'Rank', 'Name', 'Price', '1h %', '24h %', '7d %', 'Market Cap', 'Volume(24h)', 'Circulating Supply', 'Last 7 Days'
])

# Remover a primeira coluna vazia (se presente)
df = df.drop(columns=[''])

# Limpeza de dados
df.dropna(inplace=True)

# Conectar ao PostgreSQL
conn = psycopg2.connect(
    dbname="webScraping",  
    user="postgres",      
    password="312619",    
    host="localhost",     
    port="5432"
)

cur = conn.cursor()

# Criar a tabela no PostgreSQL 
create_table_query = """
CREATE TABLE IF NOT EXISTS public.coinmarketcap_data (
    id SERIAL PRIMARY KEY,  -- Isso cria uma coluna 'id' que vai ser auto-incrementada
    rank TEXT,
    name TEXT,
    price TEXT,
    hour_change TEXT,
    day_change TEXT,
    week_change TEXT,
    market_cap TEXT,
    volume_24h TEXT,
    circulating_supply TEXT,
    last_7_days TEXT
);
"""
cur.execute(create_table_query)
conn.commit()

# Inserir os dados no PostgreSQL
for index, row in df.iterrows():
    # Converte os dados da linha para string
    values = [str(x) for x in row.values]  # Converte os valores para string (sem incluir o índice)

    # Verificar se o número de valores é 10, caso contrário, imprimir um alerta
    if len(values) != 10:
        print(f"Erro nos dados da linha {index}: {values}")
        continue  # Pula para a próxima linha se o número de valores não for o esperado

    # Consulta SQL de inserção
    insert_query = """
    INSERT INTO coinmarketcap_data (
        rank, name, price, hour_change, day_change, week_change, market_cap, volume_24h, circulating_supply, last_7_days
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    
    # Executar a inserção
    cur.execute(insert_query, values)

# Confirmar as alterações no banco de dados
conn.commit()

# Fechar a conexão
cur.close()
conn.close()
print("Dados salvos com sucesso no banco de dados PostgreSQL!")


In [ ]:
df.head(5)

# Consumindo API WeatherAPI
### Chave api: 65da5addbebb488e908175307240412 

In [ ]:
#Importando bibliotecas necessarias 
import requests
import pandas as pd
import psycopg2
from psycopg2  import sql

In [ ]:
#chave da api 
API_KEY = '65da5addbebb488e908175307240412'

#URL API
url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q=mexico"

#Requisição da api
response = requests.get(url)
data = response.json()

#Fazendo a extração dos dados 

weather_data = {
    'location': data['location']['name'],
    'region': data['location']['region'],
    'country': data['location']['country'],
    'temperature_c': data['current']['temp_c'],
    'temperature_f': data['current']['temp_f'],
    'humidity': data['current']['humidity'],
    'wind_kph': data['current']['wind_kph'],
    'condition': data['current']['condition']['text'],
    'last_updated': data['current']['last_updated']
} 


#Passando dados para DF

df = pd.DataFrame([weather_data])
#print(df)

#Conectar ao POSTGRESQL
db_params = {
    'dbname': 'weather_data',
    'user': 'postgres',
    'password': '312619',
    'host': 'localhost',  # ou outro host, dependendo do seu setup
    'port': '5432'
}

# Conectar ao banco de dados PostgreSQL
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

#Criando a tabela no banco 
create_table_query = """
CREATE TABLE IF NOT EXISTS weather_data (
    id SERIAL PRIMARY KEY,
    location VARCHAR(255),
    region VARCHAR(255),
    country VARCHAR(255),
    temperature_c FLOAT,
    temperature_f FLOAT,
    humidity INT,
    wind_kph FLOAT,
    condition VARCHAR(255),
    last_updated TIMESTAMP
);
"""

cursor.execute(create_table_query)
conn.commit()

#Inserção dos dados na tabela 
insert_query = sql.SQL("""
INSERT INTO weather_data (location, region, country, temperature_c, temperature_f, humidity, wind_kph, condition, last_updated)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

# Convertendo a data para o formato adequado
insert_values = (
    str(df['location'][0]),  # Convertendo para string
    str(df['region'][0]),    # Convertendo para string
    str(df['country'][0]),   # Convertendo para string
    float(df['temperature_c'][0]),  # Convertendo para float
    float(df['temperature_f'][0]),  # Convertendo para float
    int(df['humidity'][0]),  # Convertendo para inteiro
    float(df['wind_kph'][0]),  # Convertendo para float
    str(df['condition'][0]),  # Convertendo para string
    pd.to_datetime(df['last_updated'][0]).strftime('%Y-%m-%d %H:%M:%S')  # Convertendo para string de data
)

cursor.execute(insert_query, insert_values)
conn.commit()

# Fechar a conexão com o banco de dados
cursor.close()
conn.close()

print("Dados salvos no banco de dados com sucesso!")

In [ ]:
df.head(5)

# IBGE Indicadores

In [ ]:
#Importando bibliotecas necessarias
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import psycopg2

In [6]:
# Configuração do Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Se quiser rodar sem abrir o navegador
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.ibge.gov.br/indicadores.html")

# Aguarda a página carregar
driver.implicitly_wait(10)

# Extrai o conteúdo da página
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# Encontra a tabela
table = soup.find('table')

data = []
rows = table.find_all('tr')  # Encontrar todas as linhas da tabela

# Itera sobre cada linha da tabela
for row in rows:
    cols = row.find_all('td')  # Encontrar todas as células da linha
    cols = [ele.text.strip() for ele in cols]  # Remover espaços extras

    if cols:  # Só adicionar linhas que têm dados
        cleaned_row = []
        
        # Pega o nome do indicador (primeira célula da linha)
        indicador = row.find_previous('th').text.strip() if row.find_previous('th') else ""
        
        # Adiciona o nome do indicador à linha
        cleaned_row.append(indicador)
        
        # Agora processa as outras colunas
        for col in cols:
            # Usando regex para extrair apenas os números
            cleaned_col = re.sub(r'[^\d,\.]', '', col)  # Remove tudo, exceto números e vírgulas/pontos
            cleaned_col = re.sub(r'\s+', ' ', cleaned_col).strip()  # Remove espaços extras
            cleaned_row.append(cleaned_col)
        
        # Adiciona a linha ao conjunto de dados
        data.append(cleaned_row)


# Ajustar as colunas conforme necessário
columns = ['Indicador', 'Último', 'Anterior', '12 meses', 'No ano']

# Criar o DataFrame 
df = pd.DataFrame(data, columns=columns)

# Limpeza de dados (remover linhas vazias)
df.dropna(inplace=True)

# Conectar ao PostgreSQL
conn = psycopg2.connect(
    dbname="ibge",  
    user="postgres",      
    password="312619",    
    host="localhost",     
    port="5432"
)

cur = conn.cursor()

# Criar a tabela no PostgreSQL, se não existir
create_table_query = """
CREATE TABLE IF NOT EXISTS public.ibge (
    id SERIAL PRIMARY KEY,
    indicador TEXT,
    ultimo TEXT,
    anterior TEXT,
    doze_meses TEXT,
    no_ano TEXT
);
"""
cur.execute(create_table_query)
conn.commit()

# Inserir os dados no PostgreSQL
for index, row in df.iterrows():
    values = [str(x) for x in row.values]  # Converte os valores para string (sem incluir o índice)

    # Verificar se o número de valores é 5 (pois temos 5 colunas)
    if len(values) != 5:
        print(f"Erro nos dados da linha {index}: {values}")
        continue  # Pula para a próxima linha se o número de valores não for o esperado

    # Consulta SQL de inserção
    insert_query = """
    INSERT INTO ibge (
        indicador, ultimo, anterior, doze_meses, no_ano
    ) VALUES (%s, %s, %s, %s, %s);
    """
    
    # Executar a inserção
    cur.execute(insert_query, values)

# Confirmar as alterações no banco de dados
conn.commit()

# Fechar a conexão
cur.close()
conn.close()
print("Dados salvos com sucesso no banco de dados PostgreSQL!")

Dados salvos com sucesso no banco de dados PostgreSQL!


In [7]:
df.head(5)

,Indicador,Último,Anterior,12 meses,No ano
0,No ano,"0,562024","0,442024","124,76","3,88"
2,IPCA (%),"0,612024","0,482024","124,60","3,92"
4,INPC (%),"0,622024","0,542024","124,77","4,35"
6,IPCA-15 (%),"0,942024","0,622024","125,89","6,46"
8,IPP (%),"0,532024","0,352024","123,86","3,51"
